In [ ]:
!pip3 uninstall tensorflow_datasets
!pip3 install git+git://github.com/sputney13/datasets

In [ ]:
import tensorflow
import tensorflow_datasets as tfds
import numpy
import os
import matplotlib
import sklearn

from trainer.models import unet, patchGAN
from trainer.utils import edges_2_shoes, cityscapes, oasbud, training_utils

## Edges2Shoes Image Generation

In [ ]:
_, example_data = edges_2_shoes.create_dataset(4)

In [ ]:
shoes_generator = tf.keras.models.load_model('trained_models/edge_2_shoe_generator.h5')

In [ ]:
for input_image, target in example_data.take(5):
    training_utils.generate_images(shoes_generator, input_image, target)

## Cityscapes Image Generation

In [ ]:
_, example_data = cityscapes.create_dataset(1)

In [ ]:
cityscape_generator = tf.keras.models.load_model('trained_models/cityscapes_generator.h5')

In [ ]:
for input_image, target in example_data.take(5):
    training_utils.generate_images(cityscape_generator, input_image, target)

## OASBUD Image Generation

In [ ]:
dataset = tfds.load('oasbud/b_mode') # not yet merged into Tensorflow Datasets, must install @sputney13 git version

In [ ]:
bmode_ds = oasbud.process_oasbud_for_gan(dataset)

In [ ]:
oasbud_generator = tf.keras.models.load_model('trained_models/oasbud_generator.h5')

In [ ]:
for input_image, target in bmode_ds.take(5):
    training_utils.generate_images(oasbud_generator, input_image, target)

## OASBUD Classification

#### Without Augmentation

In [ ]:
bmode_train, bmode_test = oasbud.process_oasbud_for_classification(dataset)

In [ ]:
## THE FOLLOWING MODEL WAS TRAINED USING:

# inputs = tf.keras.Input(shape=[1024, 512, 1])
# efficient_net = tf.keras.applications.EfficientNetB0(weights = None, input_tensor=inputs, classes = 2)(inputs)
# model = tf.keras.Model(inputs, efficient_net)
# model.compile(optimizer="adam", loss="binary_crossentropy")
# model.fit(bmode_train, epochs=20, validation_data=bmode_test)

In [ ]:
efficient_net = tf.keras.models.load_model('trained_models/efficientnet_no_aug.h5')

In [ ]:
X_test = bmode_test.map(lambda X_test, y_test: X_test)
y_test = bmode_test.map(lambda X_test, y_test: y_test)

In [ ]:
labels = [tf.argmax(x).numpy() for x in y_test]
preds = [tf.argmax(x).numpy() for x in efficient_net.predict(X_test)]
accuracy = sum(np.equals(labels, preds)) / len(y_test)
print(accuracy)

#### With Augmentation

In [ ]:
benign_generator = tf.keras.models.load_model('trained_models/benign_generator.h5')
malignant_generator = tf.keras.models.load_model('trained_models/malignant_generator.h5')

In [ ]:
bmode_train, bmode_test = oasbud.process_oasbud_for_classification_with_aug(dataset, benign_generator, malignant_generator)

In [ ]:
## THE FOLLOWING MODEL WAS TRAINED USING:

# inputs = tf.keras.Input(shape=[1024, 512, 1])
# efficient_net = tf.keras.applications.EfficientNetB0(weights = None, input_tensor=inputs, classes = 2)(inputs)
# model = tf.keras.Model(inputs, efficient_net)
# model.compile(optimizer="adam", loss="binary_crossentropy")
# model.fit(bmode_train, epochs=20, validation_data=bmode_test)

In [ ]:
efficient_net = tf.keras.models.load_model('trained_models/efficient_net_aug.h5')

In [ ]:
X_test = bmode_test.map(lambda X_test, y_test: X_test)
y_test = bmode_test.map(lambda X_test, y_test: y_test)

In [ ]:
labels = [tf.argmax(x).numpy() for x in y_test]
preds = [tf.argmax(x).numpy() for x in efficient_net.predict(X_test)]
accuracy = sum(np.equals(labels, preds)) / len(y_test)
print(accuracy)

## OASBUD Segmentation

#### Without Augmentation

In [ ]:
bmode_train, bmode_test = oasbud.process_oasbud_for_segmentation(dataset)

In [ ]:
## THE FOLLOWING MODEL WAS TRAINED USING:

# unet = unet.UNet(output_channels = 2)
# unet.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
# unet.fit(bmode_train, epochs=6, validation_data=bmode_test)

In [ ]:
unet = tf.keras.models.load_model('trained_models/unet_no_aug.h5')

In [ ]:
for input_image, mask in bmode_test.take(5):
    training_utils.generate_images(unet, input_image, mask)

#### With Augmentation

In [ ]:
oasbud_generator = tf.keras.models.load_model('trained_models/oasbud_generator.h5')

In [ ]:
bmode_train, bmode_test = oasbud.process_oasbud_for_segmentation_with_aug(dataset, oasbud_generator)

In [ ]:
## THE FOLLOWING MODEL WAS TRAINED USING:

# unet = unet.UNet(output_channels = 2)
# unet.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
# unet.fit(bmode_train, epochs=6, validation_data=bmode_test)

In [ ]:
unet = tf.keras.models.load_model('trained_models/unet_aug.h5')

In [ ]:
for input_image, mask in bmode_test.take(5):
    training_utils.generate_images(unet, input_image, mask)